In [ ]:
!git remote add origin git@github.com:gprakash/supplychaingraph.git
#git branch -M main
#git push -u origin main

In [ ]:
!pip install -q langchain
!pip install -q google-cloud-aiplatform>=1.35.0
!pip install -q neo4j
!pip install -q --upgrade gradio
!pip install -q --upgrade typing_extensions

In [ ]:

#@title Setting up the Auth
import os

#Authentication with Google
import sys
if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

import gradio as gr

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.llms import VertexAI
from langchain.prompts.prompt import PromptTemplate


from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai

In [ ]:
vertexai.init(project="argolis-project-340214")
# %%

llm = VertexAI()
print(llm("What is the future of supply chain in an LLM world?"))

In [ ]:

from neo4j import GraphDatabase

In [ ]:
#import getpass

# You will need to change these variables
connectionUrl = "bolt://34.173.213.251:7687"
username = "neo4j"
password = "password"

In [ ]:
# You will need to change these variables
connectionUrl = "neo4j+s://f28aa3c1.databases.neo4j.io:7687"
username = "neo4j"
password = "BAZuPrEv4CexTEzucDbJt019MuwfX91kVWv1lRaGNcA"

In [ ]:
driver = GraphDatabase.driver(connectionUrl, auth=(username, password))
driver.verify_connectivity()

In [ ]:
import pandas as pd
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
run_query("match (n:Location) return n.locationKey")

In [ ]:
run_query("match (n) return labels(n) as label, properties(n) as properties")

In [ ]:
graph = Neo4jGraph(
    url=connectionUrl, username=username, password=password
)

In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True
)

In [ ]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# What are all the parent items and where are they produced?
MATCH (il:ItemLocation)-[:CONTAINS]->(:ItemLocation) where NOT (:ItemLocation)-[:CONTAINS]->(il)
MATCH (il)-[:MADE_AT]-(l:Location)
RETURN distinct il.itemNumber AS parentItem, l.locationDescription AS productionLocation
# What products does the Seattle supplier make?
MATCH (il)-[:MADE_AT]-(l:Location)
WHERE l.locationDescription contains 'Seattle'
RETURN il.itemNumber
# Name all suppliers for Transmission?
MATCH (il)-[:MADE_AT]->(l:Location)
WHERE il.itemNumber contains 'Transmission'
RETURN l.locationDescription
# Name all suppliers for Engines?
MATCH (il)-[:MADE_AT]->(l:Location)
WHERE il.itemNumber contains 'Engine'
RETURN l.locationDescription
# Name all the suppliers for the WV plant
MATCH (plant:Location)<-[:MADE_AT]-(parentItemLocation:ItemLocation)-[:CONTAINS]->(childItemLocation:ItemLocation)-[:MADE_AT]->(supp:Location)
WHERE plant.locationDescription contains 'WV'
RETURN supp.locationId
# Name all the suppliers for the Xdock
MATCH (plant:Location)<-[:MADE_AT]-(parentItemLocation:ItemLocation)-[:CONTAINS]->(childItemLocation:ItemLocation)-[:MADE_AT]->(supp:Location)
WHERE plant.locationDescription contains 'XDock' or plant.locationDescription contains 'Xdock'
RETURN supp.locationId
# What products does Guduvan supply
MATCH (supplier:Location)-[:MAKES]->(i:Item)
WHERE supplier.locationId contains 'Guduvan'
RETURN i.itemNumber, i.itemDescription, i.itemIndicator
# Name all suppliers of StoresDC and their corresponding items
MATCH (l:Location)<-[:MADE_AT]-(il:ItemLocation)-[:CONTAINS]->(child:ItemLocation)-[MADE_AT]->(supp:Location) where l.locationDescription contains 'StoresDC'
RETURN supp.locationId, child.itemNumber
# How many units of Cups do I need to make 1 unit of Body Bra at StoresDC
match p=(top:ItemLocation)-[:CONTAINS*1..9]->(child:ItemLocation), q=(top)-[:MADE_AT]->(l:Location) where top.itemLocationDescription contains 'BodyBra' and l.locationDescription contains 'StoresDC' and child.itemLocationDescription contains 'cups'
return reduce(LineageQty = 1, y in relationships(p)| LineageQty *  toInteger(y.quantity)) as LineageQty
# Name all the suppliers of Fabric for the DirectDC
MATCH (destination:Location)<-[:MADE_AT]-(parent:ItemLocation)-[:CONTAINS]->(child:ItemLocation)-[:MADE_AT]->(source:Location)
WHERE child.itemLocationDescription contains 'Fabric' AND destination.locationDescription contains 'DirectDC'
RETURN distinct source.locationId
The question is:
{question}"""


CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatVertexAI(temperature=0), graph=graph, verbose=True, cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [ ]:
!pip install graphistry

In [ ]:
dir(graphistry)

In [ ]:
import graphistry
GRAPHISTRY_API_KEY='R4YK24YSU3SVUFRI'
#graphistry.register(api=3, username='abc', password='xyz')  # Free: hub.graphistry.com
graphistry.register(api=3, personal_key_id='ZKJQM2BSNK', personal_key_secret=GRAPHISTRY_API_KEY)
NEO4J_CREDS = {'uri': connectionUrl, 'auth': (username, password)}
graphistry.register(bolt=NEO4J_CREDS)
graphistry.cypher("MATCH (n1)-[r1]->(n2) RETURN n1, r1, n2 LIMIT 1000").plot()
import plotly.graph_objects as go
fig = go.Figure()


In [ ]:
chain.run("What are all the parent items and where are they produced?")

In [ ]:
chain.run("What all locations are there?")

In [ ]:
chain.run("What all itemlocations are there for the itemgoverning system SAP2 and print the list of item Location Descriptions?")

In [ ]:
chain.run("What all top level items are being produced and where")

In [ ]:
chain.run("What products does the Seattle supplier make")

In [ ]:
chain.run("What products does Stretch Fabrics make")

In [ ]:
chain.run("Name all the suppliers for Fabric")

In [ ]:
#chain.run("Name all the suppliers for the WV plant")
chain.run("Name all the suppliers for the Xdock")

In [ ]:
#chain.run("Name all the suppliers of Engines for the WV plant")
chain.run("Name all the suppliers of Fabric for the DirectDC")

In [ ]:

def chatbot(inputtext):
    return chain.run(inputtext)


iface = gr.Interface(fn=chatbot,
                      examples=[
#                               "Name all the suppliers for the WV plant",
#                               "Name all the suppliers for Engines",
#                               "Name all the suppliers of Engines for the WV plant",
#                               "What all top level items are being produced and where",
                               "Name all the suppliers for the Xdock",
                               "Name all the suppliers for StoresDC",
                               "Name all the suppliers for DirectDC and their corresponding items",
                               "Name all the suppliers of Fabric for the DirectDC",
                               "What all top level items are being produced and where",
                              "How many units of Cups do I need to make 1 unit of Body Bra at DirectDC"],
                      title="Victoria's Secret: Supply Chain Analyst",
                      inputs=gr.Textbox(),
                      outputs=[gr.Textbox()])

iface.launch(debug=True)



In [ ]:
graph.refresh_schema()

In [ ]:
!pip install --quiet --upgrade graphdatascience==1.0.0

In [ ]:
import pandas as pd
from graphdatascience import GraphDataScience

# If you are connecting the client to an AuraDS instance, you can get the recommended non-default configuration settings of the Python Driver applied automatically. To achieve this, set the constructor argument aura_ds=True
gds = GraphDataScience(connectionUrl, auth=(username, password), aura_ds=True)

gds.set_database("neo4j")

In [ ]:

# node labels
result = gds.run_cypher(
    """
CALL db.labels() YIELD label
CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq', {})
YIELD value
RETURN label, value.freq AS freq
"""
)

display(result)

#print(graph.schema)

In [ ]:

with gr.Blocks() as demo:
  with gr.Row():
    gr.Markdown(
    """
    # Victoria's Secret - Multi-Tier Supply Network Model Questions
    Provide the query about your supply chain model
    """)

  with gr.Row():
    with gr.Column():
      Query = gr.Textbox(label = "Type your query here")
      examples = ["What all top level items are being produced and where","What all locations are there?","What all itemlocations are there for the itemgoverning system SAP2 and print the list of item Location Descriptions?",]
      ExampleQueries = gr.Examples(examples=examples, inputs=Query)
      getLLMResponse_btn = gr.Button("Get LLM Response")

  with gr.Row():
    with gr.Column():
      LLMResponse = gr.Textbox(label = "LLM Response")


    getLLMResponse_btn.click(fn=chain.run, inputs=Query, outputs=LLMResponse)

app = demo.app

demo.launch(debug=True)


In [ ]:
%pip install yfiles_jupyter_graphs --quiet
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
w = GraphWidget()
w.nodes = [
    {"id": 0, "properties": {"firstName": "Alpha", "label": "Person A"}},
    {"id": "one", "properties": {"firstName": "Bravo", "label": "Person B"}},
    {"id": 2.0, "properties": {"firstName": "Charlie", "label": "Person C", "has_hat": False}},
    {"id": True, "properties": {"firstName": "Delta", "label": "Person D", "likes_pizza": True}}
]
w.edges = [
    {"id": "zero", "start": 0, "end": "one", "properties": {"since": "1992", "label": "knows"}},
    {"id": 1, "start": "one", "end": True, "properties": {"label": "knows", "since": "1992"}},
    {"id": 2.0, "start": 2.0, "end": True, "properties": {"label": "knows", "since": "1992"}},
    {"id": False, "start": 0, "end": 2.0, "properties": {"label": "knows", "since": 234}}
]

w.nodes =
w.directed = True

In [ ]:
w.show()  # or just w


In [ ]:
df = run_query("call apoc.meta.data()")
print(df)

In [ ]:
from py2neo import Graph
from pandas import DataFrame
# remove search by au.id='1' and limit so that you will get all
# return the id in your query
session = Graph(connectionUrl, auth=(username, password))
query = '''  MATCH (il)-[:MADE_AT]-(l:Location)  RETURN il '''
# access the result data
result = session.run(query)
#df = run_query(query)
# convert result into pandas dataframe
df = DataFrame(result)
df.head()

In [ ]:
#!pip install json2table
import json2table
import json
from json2table import convert
newdf = parse(df[0])
print(newdf)

html=convert(df[0])
print(html)